Event to action list mapping
========
Idea: map all events to a list of possible actions.

Pic a random element from the list of possible actions as the next action at an event.

If the load of the elevator increases and then drops again, our last action was a good one and we increase
the chance by adding it to the list of possible actions again

In [ ]:
import remotejupyter
if not "remote" in locals():
    remote = remotejupyter.startServer()

Saga code
=====

    {
        init: function(elevators, floors) {
            var gameState = {
                'elevatorplanning': []
            };



            $.ajaxSetup({
                url: "http://localhost:5000/",
                type: "POST",
                contentType: 'application/json; charset=UTF-8',
                processData: false,
                dataType: 'json'
            });
            function pushGameState() {
                $.ajax({data: JSON.stringify(gameState), success: function (actions) {
                    _.forEach(actions, function(action) {
                        console.log("Going to ", action);
                        eval(action);
                    })
                }});
            }
            $.ajax({data: JSON.stringify({
                'world': {'nElevators': elevators.length, 'nFloors': floors.length}
            })})
            _.forEach(elevators, function(ev, idx){
               gameState.elevatorplanning.push(new Array(floors.length)); 

               ev.on("floor_button_pressed", function(floorNum){
                   gameState.elevatorplanning[idx][floorNum] = true;
                   pushGameState();
                });
               ev.on("stopped_at_floor", function(floorNum){
                   gameState.elevatorplanning[idx][floorNum] = false; 
                   pushGameState();
               });
            });

        },
        update: function(dt, elevators, floors) {
        }
    }

In [ ]:
#List of all possible actions
actions = []
eventActionMapping = {}
actionHistory = []
loadFactorHistory = []

In [ ]:
def nub(l):
    last = None
    for el in l:
        if el != last:
            yield el
            last = el
def perThree(l):
    three = [0, 0, 0]
    for el in l:
        three.append(el)
        three.pop(0)
        yield three

In [ ]:
def lastElevatorBehaviour(loadFactors):
    '''Return the loadfactor changes of all elevators'''
    nElevators = len(loadFactors[0])
    transposed = [[row[evIdx] for row in loadFactors] for evIdx in range(nElevators)]
    def loadStateChangeClassification(history):
        for moment in perThree(nub(history)):
            pass
        if moment[1] > moment[0] and moment[2] < moment[1]:
            return 1
        return 0
        
    return map(loadStateChangeClassification, transposed)        
    
if len(loadFactorHistory):
    #Run test
    list(lastElevatorBehaviour(loadFactorHistory))

In [ ]:
def collapseEventActionMapping(uncollapsed):
    '''Remove the maximum amount of duplicates for each action in each event listing'''
    collapsedMapping = {k: [] for k in uncollapsed}
    for event, actions in uncollapsed.items():
        unique = set(actions)
        counts = {u : actions.count(u) for u in unique}
        base = min(counts.values())
        for k, v in counts.items():
            collapsedMapping[event].extend(list([k] * (1 + v - base)))
    return collapsedMapping
#collapseEventActionMapping(eventActionMapping)

In [ ]:
import random
def handleState(msg):
    global actions, eventActionMapping, loadFactorHistory
    
    if 'world' in msg:
        if len(actions) == 0:
            #Set up all possible actions
            for ev in range(0, msg['world']['nElevators']):
                for f in range(0, msg['world']['nFloors']):
                    actions.append("elevators[%i].goToFloor(%i)" % (ev, f))
        #Optimize previous eventActionMapping arrays
        eventActionMapping = collapseEventActionMapping(eventActionMapping)
        return []
    
    if 'loadFactors' in msg:
        loadFactorHistory.append(msg['loadFactors'])
        for idx, elevatorEvaluation in enumerate(lastElevatorBehaviour(loadFactorHistory)):
            if elevatorEvaluation > 0:
                for event in reversed(actionHistory):
                    if len(eventActionMapping[event[0]]) > 0 and ('elevators[%i]' % idx) in event[1]:
                        if eventActionMapping[event[0]][-1] != event[1]:
                            eventActionMapping[event[0]].append(event[1])
        return []
    
    #msg is some kind of actionable event
    msgIdent = ','.join([str(e) for e in msg])
    if not msgIdent in eventActionMapping:
        eventActionMapping[msgIdent] = list(actions)
    
    action = random.sample(eventActionMapping[msgIdent], 1)
    actionHistory.append([msgIdent, action[0]])
    return action


In [ ]:
remote.put(handleState)

In [ ]:
eventActionMapping